In [9]:
import os
os.environ["OPENAI_API_KEY"]="sk-gnIprVqai1zPT1szjDfghgfpoiouyiyiuhgfoIBLBVol0fqwertyu8AZi"

In [10]:
from langchain.llms import OpenAI

In [11]:
llm = OpenAI(temperature = 0.6)
#Higher the temperature, more creative the llm behaves but also involves more risk

In [12]:
llm2 = OpenAI(openai_api_key= os.environ["OPEN_API_KEY"], temperature= 0.6)

In [1]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]="hf_uaVVdwctrerkDYvcxvcxv534534CfXaONRhzfDtVjghjghjghhENhrYuGN"

In [5]:
from langchain import HuggingFaceHub
# llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-large",model_kwargs={"temperature":6,"max_length":64})
llm_huggingface = HuggingFaceHub(repo_id="google/flan-t5-xxl",  model_kwargs={"temperature": 0.5, "max_length": 64})
# llm_huggingface = HuggingFaceHub(repo_id="tiiuae/falcon-40b",  model_kwargs={"temperature": 0.5, "max_length": 64})

/home/pranjal/Downloads/langchain-conversational-llm/venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [9]:
text = "How to convert water to ice"
output=llm_huggingface.predict(text)
print(output)

Put water in a bowl and freeze.


Prompt Template and LLM Chain

In [48]:
#Prompt template defines how the expected input and output should be

In [36]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="What is the capital of {country}")

prompt_template.format(country="Nepal")

'What is the capital of Nepal'

In [37]:
from langchain.chains import LLMChain
chain=LLMChain(llm=llm_huggingface,prompt=prompt_template)
print(chain.run("Nepal"))

kathmandu


In [43]:
capital_template=PromptTemplate(input_variables=['country'],
template="What is the capital of the {country}")
capital_chain=LLMChain(llm=llm_huggingface,prompt=capital_template)

famous_place_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some famous places to visit in {capital}")
famous_chain=LLMChain(llm=llm_huggingface,prompt=famous_place_template)

In [44]:
from langchain.chains import SimpleSequentialChain
chain=SimpleSequentialChain(chains=[capital_chain,famous_chain])
chain.run("Nepal")

'Kathmandu Museum'

In [46]:
capital_template=PromptTemplate(input_variables=['country'],
template="What is the capital of the {country}")
capital_chain=LLMChain(llm=llm_huggingface,prompt=capital_template,output_key="capital")

famous_place_template=PromptTemplate(input_variables=['capital'],
template="Suggest me some famous places to visit in {capital}")
famous_chain=LLMChain(llm=llm_huggingface,prompt=famous_place_template,output_key="places")

In [47]:
from langchain.chains import SequentialChain
chain=SequentialChain(chains=[capital_chain,famous_chain],
input_variables=['country'],
output_variables=['capital',"places"])
chain({'country':"Nepal"})

{'country': 'Nepal', 'capital': 'kathmandu', 'places': 'Kathmandu Museum'}

ChatBot with ChatOpenAI

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [64]:
from langchain.chat_models import ChatOllama

In [65]:
chatllm= ChatOllama()

In [50]:
chatllm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6,model='gpt-3.5-turbo')

In [78]:
# chatllm([
# SystemMessage(content="Yor are a comedian AI assitant"),
# HumanMessage(content="Please provide some comedy punchlines on AI")
# ])

Promp template and LLm with output parser

In [52]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [60]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")

In [61]:
template="Your are a helpful assistant. When the user given any input , you should generate 5 words synonyms in a comma seperated list"
human_template="{text}"
chatprompt=ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [62]:
chain=chatprompt|chatllm|Commaseperatedoutput()

In [79]:
# chain.invoke({"text":"intelligent"})